# 기상청 자료

## 기상청 rss
- https://www.weather.go.kr/weather/lifenindustry/sevice_rss.jsp

## endpoint
- http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108

In [19]:
import urllib.request as req
import requests
from bs4 import BeautifulSoup
import os.path
import time

In [5]:
url = 'http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108'
dt = time.strftime('%Y%m%d', time.localtime(time.time()))
xml_file = 'forecast_{}.xml'.format(dt)

## xml 파일로 저장

In [6]:
if not os.path.exists(xml_file):
    req.urlretrieve(url, xml_file)

In [7]:
# try-catch를 해야되지만 명백히 있는 파일이니까 바로 read
xml = open(xml_file, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml)

In [9]:
location = soup.select_one('location')

In [12]:
location.select_one('city').text

'서울'

In [13]:
locations = soup.select('location')

In [16]:
info = dict()
for location in locations:
    city = location.select_one('city').text
    weather = location.select_one('wf').text
    if not (weather in info):
        info[weather] = []
    info[weather].append(city)

In [18]:
#info

# 케이웨더 생활날씨-생활지수 가져오기

In [22]:
url = 'https://www.kweather.co.kr/forecast/forecast_living_jisu.html'
response = requests.get(url)
soup = BeautifulSoup(response.text)
#soup

In [57]:
ul = soup.select_one('#jisu_navi_content')
a_href = ul.select_one('a')['href']
a_href

"javascript:startRequest('../data/JISU/11B00000.xml','1');"

In [58]:
address1 = a_href.split('..')[1]
address2 = address1.split("',")[0]
address2

'/data/JISU/11B00000.xml'

In [64]:
a_href_all = ul.select('a')
xml_list = []
for item in a_href_all:
    a_href = item['href']
    address1 = a_href.split('..')[1]
    address2 = address1.split("',")[0]
    xml_list.append(address2)

In [65]:
xml_list

['/data/JISU/11B00000.xml',
 '/data/JISU/11D10000.xml',
 '/data/JISU/11D20000.xml',
 '/data/JISU/11C10000.xml',
 '/data/JISU/11C20000.xml',
 '/data/JISU/11H10000.xml',
 '/data/JISU/11H20000.xml',
 '/data/JISU/11F10000.xml',
 '/data/JISU/11F20000.xml',
 '/data/JISU/11G00000.xml']

## 정규표현식 사용하여 가져오기

In [66]:
import re

In [68]:
xml_re_list = []
for item in a_href_all:
    a_href = item['href']
    info = re.search('/data/JISU/.+.xml', a_href)
    xml_re_list.append(info.group())

In [69]:
xml_re_list

['/data/JISU/11B00000.xml',
 '/data/JISU/11D10000.xml',
 '/data/JISU/11D20000.xml',
 '/data/JISU/11C10000.xml',
 '/data/JISU/11C20000.xml',
 '/data/JISU/11H10000.xml',
 '/data/JISU/11H20000.xml',
 '/data/JISU/11F10000.xml',
 '/data/JISU/11F20000.xml',
 '/data/JISU/11G00000.xml']

In [70]:
# 서울/경기
url = 'https://www.kweather.co.kr{}'.format(xml_list[0])

In [73]:
response = requests.get(url)

In [74]:
response.status_code

200

In [77]:
response.encoding = None

In [78]:
soup = BeautifulSoup(response.text)
soup

<?xml version='1.0' encoding='utf-8' ?>
<jisu>
<areacode>11B00000</areacode>
<areaname>서울,경기</areaname>
<tm>2020/06/08</tm>
<tmyyyy>2020</tmyyyy>
<tmmm>06</tmmm>
<tmdd>08</tmdd>
<tmyo>월</tmyo>
<ndate num="0">
<jtitle>
<name>나들이</name>
<jfix>나들이</jfix>
<num>01</num>
<jnum>80</jnum>
<jtext>더워서 걱정이네요</jtext>
</jtitle>
<jtitle>
<name>빨래</name>
<jfix>빨래</jfix>
<num>02</num>
<jnum>90</jnum>
<jtext>아주 잘 말라요</jtext>
</jtitle>
<jtitle>
<name>세차</name>
<jfix>세차</jfix>
<num>03</num>
<jnum>90</jnum>
<jtext>세차효과 길어요</jtext>
</jtitle>
<jtitle>
<name>난방</name>
<jfix>난방</jfix>
<num>04</num>
<jnum>20</jnum>
<jtext>난방 불필요해요</jtext>
</jtitle>
<jtitle>
<name>불조심</name>
<jfix>불조심</jfix>
<num>05</num>
<jnum>20</jnum>
<jtext>소화기 점검은 평소에</jtext>
</jtitle>
<jtitle>
<name>운동</name>
<jfix>운동</jfix>
<num>06</num>
<jnum>90</jnum>
<jtext>컨디션이 좋습니다</jtext>
</jtitle>
<jtitle>
<name>감기</name>
<jfix>감기</jfix>
<num>07</num>
<jnum>60</jnum>
<jtext>저항력을 높이세요</jtext>
</jtitle>
<jtitle>
<name>공해</name>
<jfix>공해</jfix>
<num>

### 데이터 한개 가져오기

In [80]:
jisu_name = soup.select_one('name').text
jisu_jnum = soup.select_one('jnum').text
jisu_jtext = soup.select_one('jtext').text
print(jisu_name, jisu_jnum, jisu_jtext)

나들이 80 더워서 걱정이네요


### 데이터 전체 가져오기

In [83]:
jisu_name_all = soup.select('name')
jisu_jnum_all = soup.select('jnum')
jisu_jtext_all = soup.select('jtext')

In [84]:
print(len(jisu_name_all))
print(len(jisu_jnum_all))
print(len(jisu_jtext_all))

42
42
42


In [87]:
jisu_sk_list = []
for i in range(len(jisu_name_all)):
    jisu_i = (jisu_name_all[i].text, jisu_jnum_all[i].text, jisu_jtext_all[i].text)
    jisu_sk_list.append(jisu_i)

In [89]:
jisu_sk_list

[('나들이', '80', '더워서 걱정이네요'),
 ('빨래', '90', '아주 잘 말라요'),
 ('세차', '90', '세차효과 길어요'),
 ('난방', '20', '난방 불필요해요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '90', '컨디션이 좋습니다'),
 ('감기', '60', '저항력을 높이세요'),
 ('공해', '60', '대체로 좋은 편'),
 ('냉방', '60', '쾌적한 실내온도 유지하세요'),
 ('불쾌', '80', '마음의 폭을 넓히세요'),
 ('자외선', '80', '장시간 야외활동은 위험해요'),
 ('모기', '90', '모기 공격 조심'),
 ('부패', '40', '냉장보관이 안전해요'),
 ('골프', '80', '라운딩에 좋은 조건입니다.'),
 ('나들이', '70', '더위먹지 않도록 주의'),
 ('빨래', '90', '아주 잘 말라요'),
 ('세차', '90', '세차하기 좋은날'),
 ('난방', '20', '난방 불필요해요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '90', '컨디션이 좋습니다'),
 ('감기', '60', '저항력을 높이세요'),
 ('공해', '60', '대체로 좋은 편'),
 ('냉방', '60', '적당한 냉방이 필요해요'),
 ('불쾌', '80', '말다툼은 금물'),
 ('자외선', '90', '장시간 야외활동은 위험해요'),
 ('모기', '40', '모기가 힘이 없어요'),
 ('부패', '60', '냉장고에 보관하세요'),
 ('골프', '80', '라운딩에 좋은 조건입니다.'),
 ('나들이', '70', '더위먹지 않도록 주의'),
 ('빨래', '70', '마르는데 지장 없어요'),
 ('세차', '90', '광택도 내보세요'),
 ('난방', '20', '난방은 필요없어요'),
 ('불조심', '20', '소화기 점검은 평소에'),
 ('운동', '70', '수분공급 충분히'),
 ('감기', 